<a href="https://colab.research.google.com/github/levi3001/CS492i_project_VinCXR_512/blob/master/Upsample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('gdrive')

Mounted at gdrive


In [ ]:
!unzip -uq gdrive/MyDrive/cs492i_project/Vin_CXR_512.zip -d Vin_CXR_512/

In [ ]:
!unzip -uq gdrive/MyDrive/cs492i_project/vin_CXR_512.zip -d vin_CXR_512/

In [ ]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path 
import os
import matplotlib.pyplot as plt
import json 
import cv2 
import torchvision
from torch import nn as nn

In [ ]:
train_meta= pd.read_csv('vin_CXR_512/train_meta.csv')
train_meta.head(10)

,image_id,dim0,dim1
0,4d390e07733ba06e5ff07412f09c0a92,3000,3000
1,289f69f6462af4933308c275d07060f0,3072,3072
2,68335ee73e67706aa59b8b55b54b11a4,2836,2336
3,7ecd6f67f649f26c05805c8359f9e528,2952,2744
4,2229148faa205e881cf0d932755c9e40,2880,2304
5,8c87779485ae5e21e25cb57e9510e149,3170,2642
6,76b23891649862f2b3b95f9bebf0a70c,2819,2545
7,ba1795ee5daae1ed415756c3f4f21b48,3408,3320
8,aa6475267e83947ee5078281a7ff3df3,3408,2597
9,051c2436c0acdb5e09d085c7e4a764f3,2728,2596


In [ ]:
list_test=os.listdir('vin_CXR_512/test')
list_test[0][:-4]

'24263689acdde18b84fc1f464c908ccf'

In [ ]:
!cp gdrive/MyDrive/cs492i_project/test_meta.csv vin_CXR_512/

In [ ]:
test_meta= pd.read_csv('vin_CXR_512/test_meta.csv')
test_meta.head(10)

,image_id,dim0,dim1
0,002a34c58c5b758217ed1f584ccbcfe9,2584,2345
1,004f33259ee4aef671c2b95d54e4be68,3028,2517
2,008bdde2af2462e86fd373a445d0f4cd,2880,2304
3,009bc039326338823ca3aa84381f17f1,2430,1994
4,00a2145de1886cb9eb88869c85d74080,2408,2136
5,00b7e6bfa4dc1fe9ddd0ce74743e38c2,3000,3000
6,011295e0bcdc7636569ab73bfdcc4450,2430,1994
7,013c169f9dad6f1f6485da961b9f7bf2,2836,2336
8,01431a2618c0ace741e4e270a37e20b9,2880,2304
9,0168eb925aa6f28a78b16134792f5d0e,2836,2336


In [ ]:
test_meta[test_meta.image_id==list_test[0][:-4]]

,image_id,dim0,dim1
385,24263689acdde18b84fc1f464c908ccf,2430,1994


In [ ]:
def pre_process(img):
    img=np.array([img,img,img])
    mean=torch.tensor([0.485, 0.456, 0.406])
    std=torch.tensor([0.229, 0.224, 0.225])
    img=torch.from_numpy(img)
    img=img.permute(1,2,0)
    img=(img-mean)/std
    img=img.permute(2,0,1)
    img=img.unsqueeze(0)
    return img

In [ ]:
!git clone https://github.com/levi3001/Yet-Another-EfficientDet-Pytorch.git

Cloning into 'Yet-Another-EfficientDet-Pytorch'...
remote: Enumerating objects: 835, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 835 (delta 40), reused 47 (delta 22), pack-reused 765
Receiving objects: 100% (835/835), 121.22 MiB | 13.65 MiB/s, done.
Resolving deltas: 100% (477/477), done.


In [ ]:
cd Yet-Another-EfficientDet-Pytorch/

/content/Yet-Another-EfficientDet-Pytorch


In [ ]:
from backbone import EfficientDetBackbone
model = EfficientDetBackbone(num_classes=14,compound_coef_backbone=3, compound_coef=0).cuda()
model.load_state_dict(torch.load("/content/gdrive/MyDrive/cs492i_project/logs/efficientdet-d0_70_31000.pth"))

<All keys matched successfully>

In [ ]:
!pip install webcolors
#!pip install efficientnet_pytorch 

In [ ]:
from efficientnet_pytorch import EfficientNet
cls_model = EfficientNet.from_name('efficientnet-b0', num_classes=14)
cls_model.load_state_dict(torch.load('efficientnet_pretrain'))
cls_model=cls_model.cuda()

In [ ]:
cls_model = torchvision.models.resnet50(pretrained=True)
cls_model.fc = nn.Linear(2048,14)
cls_model.load_state_dict(torch.load('resnet50_best.pth'))
cls_model=cls_model.cuda()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
def format_pred(labels, boxes, scores,cls_img):
    pred_strings = []
    for label, score, bbox in zip(labels, scores, boxes):
        xmin, ymin, xmax, ymax = bbox.astype(np.int64)
        if cls_img[label]>0.5:
            pred_strings.append(f"{label} {score} {xmin} {ymin} {xmax} {ymax}")
    if len(pred_strings)>0:
        return " ".join(pred_strings)
    return '14 1.0 0 0 1 1'

In [ ]:
from utils.utils import preprocess, invert_affine, postprocess, boolean_string
from efficientdet.utils import BBoxTransform, ClipBoxes
from tqdm.notebook import tqdm
ids = []
prediction_strings = []
regressBoxes = BBoxTransform()
clipBoxes = ClipBoxes()
threshold=0.05
nms_threshold=0.2
model.eval()
with torch.no_grad():
    for test_img_name in tqdm(list_test):
        id= test_img_name[:-4]
        img=plt.imread('/content/vin_CXR_512/test/'+test_img_name)
        h = test_meta[test_meta.image_id==test_img_name[:-4]].dim0
        w = test_meta[test_meta.image_id==test_img_name[:-4]].dim1
        x=pre_process(img)
        x=x.cuda()

        features, regression, classification, anchors = model(x)
        cls_img= torch.sigmoid(cls_model(x)[0])
        preds = postprocess(x,anchors, regression, classification,
                                    regressBoxes, clipBoxes,
                                    threshold, nms_threshold)
        pred_boxes=preds[0]['rois']
        num_preds= len(preds[0]['class_ids'])
        if num_preds==0:
            pred_str='14 1.0 0 0 1 1'
        else:
            h_ratio = h / 512
            w_ratio = w / 512
            pred_boxes[:, [0, 2]] *= w_ratio
            pred_boxes[:, [1, 3]] *= h_ratio
            pred_class_ids= preds[0]['class_ids']
            pred_scores=preds[0]['scores']
            pred_str=format_pred(
                    pred_class_ids, pred_boxes, pred_scores,cls_img)
        prediction_strings.append(pred_str)
        ids.append(id)


  0%|          | 0/3000 [00:00<?, ?it/s]

In [ ]:
submission_det = pd.DataFrame({'image_id':ids, 'PredictionString': prediction_strings}, columns=['image_id', 'PredictionString'])
submission_det.to_csv("/content/submission_det.csv", index=False)
submission_det

,image_id,PredictionString
0,24263689acdde18b84fc1f464c908ccf,14 1.0 0 0 1 1
1,2e7031642f0d951eda8e991c77c611c2,14 1.0 0 0 1 1
2,608614185411b42c65111af9fc737f35,14 1.0 0 0 1 1
3,ca5305844c45ccfa56bf83c4bcf8fa5a,14 1.0 0 0 1 1
4,76e2a34a8c79a83155d9e8aa66e72fed,14 1.0 0 0 1 1
...,...,...
2995,c97e54a78bab9c05ce2e04fe6c284bcd,14 1.0 0 0 1 1
2996,cc53f79260c8d490cbfbc3cd78fe8516,14 1.0 0 0 1 1
2997,9a9c1899b72972a4c1a6a540aa2941cf,14 1.0 0 0 1 1
2998,aafba37e808192258f4a1c806113d678,14 1.0 0 0 1 1


In [ ]:
submission_det['PredictionString'][0]

'14 1.0 0 0 1 1'

In [ ]:
submission_det['PredictionString'][3]

'14 1.0 0 0 1 1'

In [ ]:
!cp /content/submission_det.csv /content/gdrive/MyDrive/submission_det5.csv